In [2]:
#Imports

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

In [3]:
address = 'Bogota, D.C.'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Bogotá are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Bogotá are 4.6726615, -74.0581744671382.


In [32]:
barriosNuevo=pd.read_excel("Localidades de Bogota con coordenadas.xlsx")
barriosNuevo.drop(barriosNuevo.index[19])
barrios = barriosNuevo.drop(barriosNuevo.index[19]).copy()
barrios.head(20)

,Localidad,Latitud,Longitud
0,Usaquen,4.719722,-74.036667
1,Chapinero,4.645833,-74.063333
2,Santa Fe,4.613889,-74.078611
3,San Cristobal,4.564722,-74.083333
4,Usme,4.473100,-74.116100
5,Tunjuelito,4.582222,-74.131944
6,Bosa,4.616944,-74.190000
7,Kennedy,4.643611,-74.153333
8,Fontibon,4.678611,-74.141111
9,Engativa,4.726111,-74.100000


In [33]:
# create map of New York using latitude and longitude values
mapaBog = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, barr in zip(barrios['Latitud'], barrios['Longitud'], barrios['Localidad']):
    label = '{}'.format(barr)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapaBog)  
    
mapaBog

In [34]:
CLIENT_ID = 'CHU2V4GIOQZEWSMHJ4V3E0IZQMXYNCPDVB0A4WRSPH5NXKYM' # your Foursquare ID
CLIENT_SECRET = 'FM4FXFLE33BTMB0330V50DNDRDH4UQPGDWNNY2PUDZ2UXW32' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CHU2V4GIOQZEWSMHJ4V3E0IZQMXYNCPDVB0A4WRSPH5NXKYM
CLIENT_SECRET:FM4FXFLE33BTMB0330V50DNDRDH4UQPGDWNNY2PUDZ2UXW32


In [37]:
#Finding nearby venues for the 18 Boroughs

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 900 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [38]:
results = requests.get(url).json()


In [12]:
# Used the function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.shape

(84, 4)

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=900):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
bogotaVenues = getNearbyVenues(names=barrios['Localidad'],
                                   latitudes=barrios['Latitud'],
                                   longitudes=barrios['Longitud']
                                  )


Usaquen
Chapinero
Santa Fe
San Cristobal
Usme
Tunjuelito
Bosa
Kennedy
Fontibon
Engativa
Suba
Barrios Unidos
Teusaquillo
Los Martires
Antonio Nariño
Puente Aranda
La Candelaria
Rafael Uribe Uribe
Ciudad Bolivar


In [42]:
print(bogotaVenues.shape)
bogotaVenues.head()

(508, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquen,4.719722,-74.036667,Wok,4.719637,-74.037489,Asian Restaurant
1,Usaquen,4.719722,-74.036667,Carulla,4.719386,-74.035684,Supermarket
2,Usaquen,4.719722,-74.036667,Juan Valdez Café,4.720062,-74.038130,Café
3,Usaquen,4.719722,-74.036667,Entre Panas y Parceros,4.720262,-74.037663,Restaurant
4,Usaquen,4.719722,-74.036667,Oma,4.719973,-74.037958,Café


In [44]:
print('There are {} uniques categories.'.format(len(bogotaVenues['Venue Category'].unique())))

There are 137 uniques categories.


In [45]:
# Finding the hottest spots for each borough
hotties = pd.get_dummies(bogotaVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hotties['Localidad'] = bogotaVenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hotties.columns[-1]] + list(hotties.columns[:-1])
hotties = hotties[fixed_columns]

hotties.head(10)

,Localidad,Advertising Agency,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Big Box Store,Bike Rental / Bike Share,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Station,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gastropub,Gay Bar,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hockey Field,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Nature Preserve,Neighborhood,Nightclub,Nightlife Spot,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pub,Public Art,Recreation Center,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court,Wings Joint
0,Usaquen,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Usaquen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Usaquen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Usaquen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Usaquen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Usaquen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Usaquen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Usaquen,0,0,0,0,0,0,

In [46]:
manhattan_onehot.shape


(508, 138)

In [47]:
#Grouping locations
gruposBog = hotties.groupby('Localidad').mean().reset_index()
gruposBog

,Localidad,Advertising Agency,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Big Box Store,Bike Rental / Bike Share,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Station,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gastropub,Gay Bar,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hockey Field,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Nature Preserve,Neighborhood,Nightclub,Nightlife Spot,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pub,Public Art,Recreation Center,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court,Wings Joint
0,Antonio Nariño,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074074,0.00,0.000000,0.00,0.00,0.074074,0.00,0.000000,0.037037,0.074074,0.000000,0.000000,0.037037,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.037037,0.000000,0.00,0.037037,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.037037,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.037037,0.000000,0.000000,0.074074,0.0000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.00,0.037037,0.000000,0.000000,0.00,0.000000,0.00000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.037037,0.074074,0.00,0.037037,0.037037,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
1,Barrios Unidos,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.035714,0.107143,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.071429,0.035714,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.035714,0.000000,0.000000,0.071429,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.035714,0.0,0.035714,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.035714,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.035714,0.00,0.00,0.00,0.00,0.107143,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.035714,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.071429,0.071429,0.000000,0.00,0.000000,0.000000,0.000000,0.000

In [50]:
gruposBog.shape


(19, 138)

In [51]:
num_top_venues = 5

for hood in gruposBog['Localidad']:
    print("----"+hood+"----")
    temp = gruposBog[gruposBog['Localidad'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Antonio Nariño----
               venue  freq
0         Restaurant  0.11
1          BBQ Joint  0.11
2   Department Store  0.07
3    Motorcycle Shop  0.07
4  Convenience Store  0.07


----Barrios Unidos----
                venue  freq
0  Mexican Restaurant  0.11
1              Bakery  0.11
2        Burger Joint  0.07
3          Donut Shop  0.07
4                Café  0.07


----Bosa----
                  venue  freq
0                  Park   0.4
1         Historic Site   0.2
2  Gym / Fitness Center   0.2
3           Supermarket   0.2
4    Advertising Agency   0.0


----Chapinero----
                           venue  freq
0                     Restaurant  0.07
1                           Café  0.06
2                      Nightclub  0.05
3                            Bar  0.05
4  Vegetarian / Vegan Restaurant  0.04


----Ciudad Bolivar----
                venue  freq
0         Coffee Shop   1.0
1  Advertising Agency   0.0
2        Neighborhood   0.0
3     Motorcycle Shop   0.0
4       

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Localidad']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Localidad'] = manhattan_grouped['Localidad']

for ind in np.arange(gruposBog.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(gruposBog.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Localidad,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Antonio Nariño,Restaurant,BBQ Joint,Motorcycle Shop,Clothing Store,Convenience Store,Department Store,Seafood Restaurant,Food Court,Shopping Mall,Smoothie Shop
1,Barrios Unidos,Bakery,Mexican Restaurant,Sandwich Place,Donut Shop,Burger Joint,Café,Restaurant,Public Art,Department Store,Burrito Place
2,Bosa,Park,Historic Site,Gym / Fitness Center,Supermarket,Department Store,Farmers Market,Electronics Store,Donut Shop,Diner,Dessert Shop
3,Chapinero,Restaurant,Café,Nightclub,Bar,Italian Restaurant,Bakery,Vegetarian / Vegan Restaurant,Latin American Restaurant,Caribbean Restaurant,Hotel
4,Ciudad Bolivar,Coffee Shop,Wings Joint,Department Store,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop,Diner,Dessert Shop,Deli / Bodega
5,Engativa,Clothing Store,Pet Store,Grocery Store,Arts & Crafts Store,Nature Preserve,Deli / Bodega,Electronics Store,Donut Shop,Diner,Dessert Shop
6,Fontibon,Bakery,Park,Food & Drink Shop,Shopping Mall,BBQ Joint,Seafood Restaurant,Restaurant,Dessert Shop,Mexican Restaurant,Gym
7,Kennedy,Coffee Shop,Fast Food Restaurant,Shopping Mall,Donut Shop,Movie Theater,Fruit & Vegetable Store,Bakery,BBQ Joint,Convenience Store,Gym
8,La Candelaria,Café,Restaurant,History Museum,Italian Restaurant,Coffee Shop,Hostel,Latin American Restaurant,Breakfast Spot,Plaza,Mexican Restaurant
9,Los Martires,Shopping Mall,Clothing Store,Train Station,Latin American Restaurant,IT Services,Bowling Green,Public Art,Mexican Restaurant,Burger Joint,Sporting Goods Shop


In [57]:
# set number of clusters
kclusters = 4

manhattan_grouped_clustering = manhattan_grouped.drop('Localidad', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:19] 

array([0, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int32)

In [58]:

# Merging data

bogoMerged = barrios
# add clustering labels
bogoMerged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bogoMerged = bogoMerged.join(neighborhoods_venues_sorted.set_index('Localidad'), on='Localidad')

bogoMerged.head(20) # check the last columns!

,Localidad,Latitud,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Usaquen,4.719722,-74.036667,0,Fast Food Restaurant,Burger Joint,Bakery,Restaurant,Pizza Place,Café,Arepa Restaurant,Asian Restaurant,Dessert Shop,Sushi Restaurant
1,Chapinero,4.645833,-74.063333,0,Restaurant,Café,Nightclub,Bar,Italian Restaurant,Bakery,Vegetarian / Vegan Restaurant,Latin American Restaurant,Caribbean Restaurant,Hotel
2,Santa Fe,4.613889,-74.078611,2,Motorcycle Shop,Farmers Market,Seafood Restaurant,Restaurant,Coffee Shop,Soccer Field,Latin American Restaurant,Department Store,Mobile Phone Shop,Monument / Landmark
3,San Cristobal,4.564722,-74.083333,0,Clothing Store,Bar,Grocery Store,Pizza Place,Playground,Fried Chicken Joint,Bakery,Donut Shop,Diner,Dessert Shop
4,Usme,4.473100,-74.116100,3,Southern / Soul Food Restaurant,Café,Wings Joint,Dessert Shop,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop,Diner,Department Store
5,Tunjuelito,4.582222,-74.131944,0,Burger Joint,Shopping Mall,Fast Food Restaurant,Gym,Cajun / Creole Restaurant,Multiplex,Department Store,Donut Shop,Diner,Dessert Shop
6,Bosa,4.616944,-74.190000,0,Park,Historic Site,Gym / Fitness Center,Supermarket,Department Store,Farmers Market,Electronics Store,Donut Shop,Diner,Dessert Shop
7,Kennedy,4.643611,-74.153333,0,Coffee Shop,Fast Food Restaurant,Shopping Mall,Donut Shop,Movie Theater,Fruit & Vegetable Store,Bakery,BBQ Joint,Convenience Store,Gym
8,Fontibon,4.678611,-74.141111,0,Bakery,Park,Food & Drink Shop,Shopping Mall,BBQ Joint,Seafood Restaurant,Restaurant,Dessert Shop,Mexican Restaurant,Gym
9,Engativa,4.726111,-74.100000,0,Clothing Store,Pet Store,Grocery Store,Arts & Crafts Store,Nature Preserve,Deli / Bodega,Electronics Store,Donut Shop,Diner,Dessert Shop


In [59]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitud'], manhattan_merged['Longitud'], manhattan_merged['Localidad'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters